In [1]:
from lib.model_utils import preprocess_features, train_and_evaluate_model, plot_forecast, forecast_bias
from lib.utils import format_plot
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import root_mean_squared_error as rmse
from sklearn.metrics import mean_squared_error as mse
import pandas as pd
import plotly.express as px
from pathlib import Path
from darts.timeseries import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, TFTModel, NBEATSModel, TSMixerModel
import torch

/home/hien/Work/AI/london_smart_meters/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%cd ../..
project_folder = Path("AI/london_smart_meters")
source_data = project_folder/"data"
preprocessed_data = project_folder/"output"/"preprocessed_data"
dst_img = project_folder/"output"/"img"
dst_result = project_folder/"output"/"result"

/home/hien/Work


In [3]:
train_df = pd.read_parquet(preprocessed_data/"selected_blocks_train_missing_imputed_feature_engg.parquet")
val_df = pd.read_parquet(preprocessed_data/"selected_blocks_val_missing_imputed_feature_engg.parquet")
test_df = pd.read_parquet(preprocessed_data/"selected_blocks_test_missing_imputed_feature_engg.parquet")

In [4]:
train_df = pd.concat([train_df, val_df])
del val_df

In [5]:
train_df

,timestamp,LCLid,frequency,timestamp_Week,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,timestamp_Is_quarter_start,timestamp_Is_year_end,timestamp_Is_year_start,timestamp_Is_month_start,timestamp_Day,timestamp_Dayofweek,timestamp_Dayofyear,timestamp_Hour,timestamp_Minute,timestamp_Elapsed
4437792,2012-01-01 00:00:00,MAC000061,30min,52,37872,Std,ACORN-Q,Adversity,block_96,NO_HOLIDAY,...,1,0,1,1,1,6,1,0,0,1325376000
4437793,2012-01-01 00:30:00,MAC000061,30min,52,37872,Std,ACORN-Q,Adversity,block_96,NO_HOLIDAY,...,1,0,1,1,1,6,1,0,30,1325377800
4437794,2012-01-01 01:00:00,MAC000061,30min,52,37872,Std,ACORN-Q,Adversity,block_96,NO_HOLIDAY,...,1,0,1,1,1,6,1,1,0,1325379600
4437795,2012-01-01 01:30:00,MAC000061,30min,52,37872,Std,ACORN-Q,Adversity,block_96,NO_HOLIDAY,...,1,0,1,1,1,6,1,1,30,1325381400
4437796,2012-01-01 02:00:00,MAC000061,30min,52,37872,Std,ACORN-Q,Adversity,block_96,NO_HOLIDAY,...,1,0,1,1,1,6,1,2,0,1325383200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25291,2014-01-31 21:30:00,MAC005529,30min,5,32688,ToU,ACORN-L,Adversity,block_82,NO_HOLIDAY,...,0,0,0,0,31,4,31,21,30,1391203800
25292,2014-01-31 22:00:00,MAC005529,30min,5,32688,ToU,ACORN-L,Adversity,block_82,NO_HOLIDAY,...,0,0,0,0,31,4,31,22,0,1391205600
25293,2014-01-31 22:30:00,MAC005529,30min,5,32688,ToU,ACORN-L,Adversity,block_82,NO_HOLIDAY,...,0,0,0,0,31,4,31,22,30,1391207400
25294,2014-01-31 23:00:00,MAC005529,30min,5,32688,ToU,ACORN-L,Adversity,block_82,NO_HOLIDAY,...,0,0,0,0,31,4,31,23,0,1391209200


In [6]:
target=["energy_consumption"]
continuous_features=[
    "visibility",
    "windBearing",
    "temperature",
    "dewPoint",
    "pressure",
    "apparentTemperature",
    "windSpeed",
    "humidity",
    "energy_consumption_lag_1",
    "energy_consumption_lag_2",
    "energy_consumption_lag_3",
    "energy_consumption_lag_4",
    "energy_consumption_lag_5",
    "energy_consumption_lag_46",
    "energy_consumption_lag_47",
    "energy_consumption_lag_48",
    "energy_consumption_lag_49",
    "energy_consumption_lag_50",
    "energy_consumption_lag_334",
    "energy_consumption_lag_335",
    "energy_consumption_lag_336",
    "energy_consumption_lag_337",
    "energy_consumption_lag_338",
    "timestamp_Elapsed",
    "day_hourly_profile",
]
categorical_features=[
    "stdorToU",
    "Acorn",
    "Acorn_grouped",
    "holidays",
    "precipType",
    "icon",
    "summary",
    "timestamp_Month",
    "timestamp_Quarter",
    "timestamp_Week",
    "timestamp_Day",
    "timestamp_Dayofweek",
    "timestamp_Dayofyear",
    "timestamp_Hour",
    "timestamp_Minute",
    "weekday",
    "hour",
]
boolean_features=[
    "timestamp_Is_quarter_end",
    "timestamp_Is_quarter_start",
    "timestamp_Is_year_end",
    "timestamp_Is_year_start",
    "timestamp_Is_month_start",
]

# Data Preprocessing

## Train

In [7]:
train_df_ts = train_df.copy()
train_df_ts["timestamp"] = pd.to_datetime(train_df_ts["timestamp"], yearfirst=True)
train_df_ts = train_df_ts.set_index(['timestamp', 'LCLid'])
lcl_ids = sorted(train_df.LCLid.unique())
for lcl_id in lcl_ids:
    lcl_id_data = train_df_ts.xs(lcl_id, level='LCLid')
    lcl_id_data = lcl_id_data.asfreq('30min')
train_df_ts = train_df_ts.drop(['frequency', 'file'], axis=1)

In [8]:
bfill_columns=[
    "pressure",
    "energy_consumption_lag_1",
    "energy_consumption_lag_2",
    "energy_consumption_lag_3",
    "energy_consumption_lag_4",
    "energy_consumption_lag_5",
    "energy_consumption_lag_46",
    "energy_consumption_lag_47",
    "energy_consumption_lag_48",
    "energy_consumption_lag_49",
    "energy_consumption_lag_50",
    "energy_consumption_lag_334",
    "energy_consumption_lag_335",
    "energy_consumption_lag_336",
    "energy_consumption_lag_337",
    "energy_consumption_lag_338",
]
n_hashed_features = 32
hashed_features_list_train = []

In [9]:
processed_train = preprocess_features(train_df_ts, continuous_features, categorical_features, boolean_features,
                                      n_hashed_features, hashed_features_list_train, bfill_columns)

Handling missing values...


/home/hien/Work/AI/london_smart_meters/lib/model_utils.py:651: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[bfill_cols] = df[bfill_cols].fillna(method="bfill")


Encoding categorical features...
Normalizing features...


## Test

In [10]:
test_df_ts = test_df.copy()
test_df_ts["timestamp"] = pd.to_datetime(test_df_ts["timestamp"], yearfirst=True)
test_df_ts = test_df_ts.set_index(['timestamp', 'LCLid'])
lcl_ids = sorted(test_df.LCLid.unique())
for lcl_id in lcl_ids:
    lcl_id_data = test_df_ts.xs(lcl_id, level='LCLid')
    lcl_id_data = lcl_id_data.asfreq('30min')
test_df_ts = test_df_ts.drop(['frequency', 'file'], axis=1)

In [11]:
hashed_features_list_test = []
processed_test = preprocess_features(test_df_ts, continuous_features, categorical_features, boolean_features,
                                      n_hashed_features, hashed_features_list_test, bfill_columns)

Handling missing values...
Encoding categorical features...


/home/hien/Work/AI/london_smart_meters/lib/model_utils.py:651: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[bfill_cols] = df[bfill_cols].fillna(method="bfill")


Normalizing features...


# Models Training

In [12]:
# Model configurations
lstm_config = {
    'name': 'LSTM',
    'class': RNNModel,
    'params': {
        'model': "LSTM",
        'input_chunk_length': 96,
        'output_chunk_length': 48,
        'training_length': 144,
        'hidden_dim': 128,
        'n_rnn_layers': 3,
        'dropout': 0.2,
        'batch_size': 32,
        'n_epochs': 2,
        'optimizer_kwargs': {'lr': 0.0001},
        'pl_trainer_kwargs': {
            'accelerator': "gpu" if torch.cuda.is_available() else "cpu",
            'devices': 1
        }
    }
}
tft_config = {
    'name': 'TFT',
    'class': TFTModel,
    'params': {
        'input_chunk_length': 96,
        'output_chunk_length': 48,
        'hidden_size': 64,
        'lstm_layers': 2,
        'num_attention_heads': 4,
        'dropout': 0.2,
        'batch_size': 32,
        'n_epochs': 5,
        'pl_trainer_kwargs': {
            'accelerator': "gpu" if torch.cuda.is_available() else "cpu",
            'devices': 1
        }
    }
}
nbeats_config = {
    'name': 'N-BEATS',
    'class': NBEATSModel,
    'params': {
        'input_chunk_length': 96,
        'output_chunk_length': 48,
        'generic_architecture': True,
        'num_stacks': 4,
        'num_blocks': 3,
        'num_layers': 4,
        'layer_widths': 128,
        'n_epochs': 5,
        'batch_size': 32,
        'pl_trainer_kwargs': {
            'accelerator': "gpu" if torch.cuda.is_available() else "cpu",
            'devices': 1
        }
    }
}
tsmixer_config = {
    'name': 'TSMixer',
    'class': TSMixerModel,
    'params': {
        'input_chunk_length': 96,
        'output_chunk_length': 48,
        'hidden_size': 64,
        'ff_size': 64,
        'num_blocks': 2,
        'activation': 'ReLU',
        'dropout': 0.2,
        'norm_type': 'LayerNorm',
        'normalize_before': False,
        'use_static_covariates': True,
        'n_epochs': 2,
        'batch_size': 32,
        'pl_trainer_kwargs': {
            'accelerator': "gpu" if torch.cuda.is_available() else "cpu",
            'devices': 1
        }
    }
}

In [13]:
# 1. Get unique household IDs
household_ids = processed_train.index.get_level_values('LCLid').unique()
print(f"Training on {len(household_ids)} households")
# 2. Initialize Scalers
target_scaler = Scaler()
covariates_scaler = Scaler()
# 3. Create empty lists to store series
train_series_list = []
train_covariates_list = []
test_series_list = []
test_covariates_list = []
all_predictions = []
all_metrics = []
ouput_predictions_tsmixer = dst_result/'tsmixer_test_predictions.parquet'
output_metrics_tsmixer = dst_result/'tsmixer_test_metrics.parquet'
ouput_predictions_lstm = dst_result/'lstm_test_predictions.parquet'
output_metrics_lstm = dst_result/'lstm_test_metrics.parquet'

Training on 185 households


In [14]:
# 4. Process each household
for i, household_id in enumerate(household_ids[:50]):

    try:
        # Extract household data
        train_household = processed_train.xs(household_id, level="LCLid")
        test_household = processed_test.xs(household_id, level="LCLid")

        # Ensure target column is first
        if 'energy_consumption' not in train_household.columns:
            print(f"Skipping household {household_id}: missing target column")
            continue

        # Move target column to first position
        cols = ['energy_consumption'] + [col for col in train_household.columns if col != 'energy_consumption']
        train_household = train_household[cols]
        test_household = test_household[cols]

        # For each household:
        test_start = test_household.index.min()
        overlap_start = test_start - pd.Timedelta(days=2)  # 48 hours before test start

        # Get overlap data from training set
        overlap_data = train_household.loc[train_household.index >= overlap_start]

        # Combine overlap with test data
        combined_test = pd.concat([overlap_data, test_household])

        # Convert to TimeSeries
        series_train = TimeSeries.from_dataframe(
            train_household,
            value_cols="energy_consumption",
            fill_missing_dates=True,
            freq="30min"
        )

        covariates_train = TimeSeries.from_dataframe(
            train_household.drop("energy_consumption", axis=1),
            fill_missing_dates=True,
            freq="30min"
        )

        series_test = TimeSeries.from_dataframe(
            test_household,
            value_cols="energy_consumption",
            fill_missing_dates=True,
            freq="30min"
        )

        covariates_test = TimeSeries.from_dataframe(
            combined_test.drop("energy_consumption", axis=1),
            fill_missing_dates=True,
            freq="30min"
        )

        # Scale series
        series_train_scaled = target_scaler.fit_transform(series_train)
        series_test_scaled = target_scaler.transform(series_test)

        # Scale covariates
        covariates_train_scaled = covariates_scaler.fit_transform(covariates_train)
        covariates_test_scaled = covariates_scaler.transform(covariates_test)

        # Add to lists
        train_series_list.append(series_train_scaled)
        train_covariates_list.append(covariates_train_scaled)
        test_series_list.append(series_test_scaled)
        test_covariates_list.append(covariates_test_scaled)

    except Exception as e:
        print(f"Error processing household {household_id}: {e}")
        continue

In [15]:
lstm_predictions, lstm_metrics = train_and_evaluate_model(
    lstm_config, train_series_list, train_covariates_list,
    test_series_list, test_covariates_list, household_ids,
    target_scaler, ouput_predictions_lstm, output_metrics_lstm
)

ignoring user defined `output_chunk_length`. RNNModel uses a fixed `output_chunk_length=1`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection |

Training LSTM model...
Epoch 1: 100%|██████████| 56925/56925 [1:05:10<00:00, 14.56it/s, train_loss=0.00253]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 56925/56925 [1:05:10<00:00, 14.56it/s, train_loss=0.00253]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  9.34it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  9.36it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  9.25it/s]

Saving final results...


In [15]:
tsmixer_predictions, tsmixer_metrics = train_and_evaluate_model(
    tsmixer_config, train_series_list, train_covariates_list,
    test_series_list, test_covariates_list, household_ids,
    target_scaler, ouput_predictions_tsmixer, output_metrics_tsmixer
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                  | Type             | Params | Mode 
-------------------------------------------------------------------
0 | criterion             | MSELoss          | 0      | train
1 | train_criterion       | MSELoss          | 0      | train
2 | val_criterion         | MSELoss          | 0      | train
3 | train_metrics         | MetricCollection | 0      | train
4 | val_metrics           | MetricCollection | 0      | train
5 | fc_hist               | Linear

Training TSMixer model...
Epoch 1: 100%|██████████| 56927/56927 [14:20<00:00, 66.16it/s, train_loss=4.97e-5] 

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 56927/56927 [14:20<00:00, 66.16it/s, train_loss=4.97e-5]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 48.05it/s]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 47.12it/s]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 49.01it/s]

Saving final results...


In [16]:
for model_name in ['lstm', 'tsmixer']:
    preds = pd.read_pickle(dst_result/f'{model_name}_test_predictions.parquet')
    metrics = pd.read_pickle(dst_result/f'{model_name}_test_metrics.parquet')
    all_predictions.append(preds)
    all_metrics.append(metrics)

combined_predictions = pd.concat(all_predictions)
combined_metrics = pd.concat(all_metrics)

In [17]:
combined_predictions

,timestamp,actual,predicted,household_id,model
0,2014-02-01 00:00:00,0.083302,0.061061,MAC000061,LSTM
1,2014-02-01 00:30:00,0.079516,0.050288,MAC000061,LSTM
2,2014-02-01 01:00:00,0.050486,0.055608,MAC000061,LSTM
3,2014-02-01 01:30:00,0.025243,0.048165,MAC000061,LSTM
4,2014-02-01 02:00:00,0.022719,0.042933,MAC000061,LSTM
...,...,...,...,...,...
64795,2014-02-27 21:30:00,0.127000,0.112165,MAC001600,TSMixer
64796,2014-02-27 22:00:00,0.113000,0.099044,MAC001600,TSMixer
64797,2014-02-27 22:30:00,0.129000,0.116393,MAC001600,TSMixer
64798,2014-02-27 23:00:00,0.085000,0.075096,MAC001600,TSMixer


In [18]:
combined_metrics

,household_id,MSE,RMSE,MAE,Forecast Bias,algorithm
0,MAC000061,0.006567,0.081040,0.054934,-0.602739,LSTM
1,MAC000062,0.007258,0.085196,0.038170,3.851821,LSTM
2,MAC000066,0.000518,0.022760,0.007899,19.028925,LSTM
3,MAC000086,0.017575,0.132572,0.096092,9.811599,LSTM
4,MAC000099,0.005212,0.072196,0.036839,1.596187,LSTM
...,...,...,...,...,...,...
45,MAC001475,0.000670,0.025880,0.013008,13.375408,TSMixer
46,MAC001511,0.000512,0.022635,0.017416,12.327882,TSMixer
47,MAC001536,0.000685,0.026180,0.018377,11.295628,TSMixer
48,MAC001589,0.001144,0.033823,0.014081,14.382056,TSMixer


In [19]:
lstm_test_metric_df = combined_predictions[combined_predictions.model =='LSTM']
overall_metrics_test_lstm = {
    "Algorithm": "LSTM",
    "MSE": mse(lstm_test_metric_df.actual.values, lstm_test_metric_df.predicted.values),
    "RMSE": rmse(lstm_test_metric_df.actual.values, lstm_test_metric_df.predicted.values),
    "MAE": mae(lstm_test_metric_df.actual.values, lstm_test_metric_df.predicted.values),
    "Forecast Bias": forecast_bias(lstm_test_metric_df.actual.values, lstm_test_metric_df.predicted.values)
}
overall_metrics_test_lstm

{'Algorithm': 'LSTM',
 'MSE': 0.010781797581043013,
 'RMSE': 0.10383543509343529,
 'MAE': 0.05251460939054514,
 'Forecast Bias': 5.513296344216006}

In [20]:
tsmixer_test_metric_df = combined_predictions[combined_predictions.model =='TSMixer']
overall_metrics_test_tsmixer = {
    "Algorithm": "TSMixer",
    "MSE": mse(tsmixer_test_metric_df.actual.values, tsmixer_test_metric_df.predicted.values),
    "RMSE": rmse(tsmixer_test_metric_df.actual.values, tsmixer_test_metric_df.predicted.values),
    "MAE": mae(tsmixer_test_metric_df.actual.values, tsmixer_test_metric_df.predicted.values),
    "Forecast Bias": forecast_bias(tsmixer_test_metric_df.actual.values, tsmixer_test_metric_df.predicted.values)
}
overall_metrics_test_tsmixer

{'Algorithm': 'TSMixer',
 'MSE': 0.0006218184046040514,
 'RMSE': 0.024936286904911314,
 'MAE': 0.0167511985896388,
 'Forecast Bias': 12.436330941461476}

## Visualization

In [ ]:
fig = px.histogram(combined_metrics,
                   x="MAE",
                   color=combined_metrics["algorithm"],
                   pattern_shape=combined_metrics["algorithm"],
                   marginal="box",
                   nbins=500,
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="mae", ylabel="Probability Density")
fig.update_layout(title="Distribution of MAE in the dataset", xaxis_range=[0,1])
fig.write_image(dst_img/"dl_mae_dist.png")
fig.show()

In [ ]:
fig = px.histogram(combined_metrics,
                   x="MSE",
                   color=combined_metrics["algorithm"],
                   pattern_shape=combined_metrics["algorithm"],
                   marginal="box",
                   nbins=500,
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="mse", ylabel="Probability Density")
fig.update_layout(title="Distribution of MSE in the dataset", xaxis_range=[0,1])
fig.write_image(dst_img/"dl_mse_dist.png")
fig.show()

In [ ]:
fig = px.histogram(combined_metrics,
                   x="RMSE",
                   color=combined_metrics["algorithm"],
                   pattern_shape=combined_metrics["algorithm"],
                   marginal="box",
                   nbins=500,
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="rmse", ylabel="Probability Density")
fig.update_layout(title="Distribution of RMSE in the dataset", xaxis_range=[0,1])
fig.write_image(dst_img/"dl_rmse_dist.png")
fig.show()

In [ ]:
fig = px.histogram(combined_metrics,
                   x="Forecast Bias",
                   color=combined_metrics["algorithm"],
                   pattern_shape=combined_metrics["algorithm"],
                   marginal="box",
                   nbins=500,
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="forecast bias", ylabel="Probability Density")
fig.update_layout(title="Distribution of Forecast Bias in the dataset", xaxis_range=[-10,20])
fig.write_image(dst_img/"dl_bias_dist.png")
fig.show()

In [22]:
display_lclid = "MAC000061"
ts_baseline_test_pred_df = combined_predictions.loc[combined_predictions.household_id==display_lclid]
ts_aggtest_metrics = combined_metrics.loc[combined_metrics.household_id==display_lclid]

In [23]:
lstm_data = ts_baseline_test_pred_df[ts_baseline_test_pred_df['model'] == 'LSTM'].copy()
tsmixer_data = ts_baseline_test_pred_df[ts_baseline_test_pred_df['model'] == 'TSMixer'].copy()

In [ ]:
## Lasso
model_name = ['LSTM']
model_display_name = ['LSTM']

fig = plot_forecast(lstm_data,
                    forecast_columns=['predicted'],
                    forecast_display_names=model_display_name, timestamp_col ='timestamp',
                    target_col_name="actual")
fig = format_plot(fig)
fig.update_layout(
    title=f"{model_name[0]}: "\
          f"MSE: {ts_aggtest_metrics.loc[ts_aggtest_metrics.algorithm==model_name[0]][['MSE']].iloc[0].item():.4f} | "\
          f"RMSE: {ts_aggtest_metrics.loc[ts_aggtest_metrics.algorithm==model_name[0]][['RMSE']].iloc[0].item():.4f} | "\
          f"MAE: {ts_aggtest_metrics.loc[ts_aggtest_metrics.algorithm==model_name[0]][['MAE']].iloc[0].item():.4f} | "\
          f"BIAS: {ts_aggtest_metrics.loc[ts_aggtest_metrics.algorithm==model_name[0]][['Forecast Bias']].iloc[0].item():.4f}"
)
fig.update_xaxes(type="date", range=["2014-02-01", "2014-02-08"])
fig.write_image(dst_img/"lstm.png")
fig.show()

In [24]:
## Lasso
model_name = ['TSMixer']
model_display_name = ['TSMixer']

fig = plot_forecast(tsmixer_data,
                    forecast_columns=['predicted'],
                    forecast_display_names=model_display_name, timestamp_col ='timestamp',
                    target_col_name="actual")
fig = format_plot(fig)
fig.update_layout(
    title=f"{model_name[0]}: "\
          f"MSE: {ts_aggtest_metrics.loc[ts_aggtest_metrics.algorithm==model_name[0]][['MSE']].iloc[0].item():.4f} | "\
          f"RMSE: {ts_aggtest_metrics.loc[ts_aggtest_metrics.algorithm==model_name[0]][['RMSE']].iloc[0].item():.4f} | "\
          f"MAE: {ts_aggtest_metrics.loc[ts_aggtest_metrics.algorithm==model_name[0]][['MAE']].iloc[0].item():.4f} | "\
          f"BIAS: {ts_aggtest_metrics.loc[ts_aggtest_metrics.algorithm==model_name[0]][['Forecast Bias']].iloc[0].item():.4f}"
)
fig.update_xaxes(type="date", range=["2014-02-01", "2014-02-08"])
fig.write_image(dst_img/"tsmixer.png")
fig.show()